In [52]:
import signalfloweeg 
import mne 

filepath = "/home/cbl/Desktop/Test_Data/0012_rest.set"
EEG = mne.io.read_raw_eeglab(filepath)

chirp_electrode_labels = {'E23', 'E18', 'E16', 'E10', 'E3', 'E28', 'E24', 'E19', 'E11', 'E4', 'E124', 'E117',
                          'E29', 'E20', 'E12', 'E5', 'E118', 'E111', 'E13', 'E6', 'E112', 'E7', 'E106'}

all_channels = EEG.ch_names

        # Identify the channels to remove
channels_to_remove = set(all_channels) - chirp_electrode_labels

        # Drop the unwanted channels from the raw data
EEG.drop_channels(channels_to_remove)

print(f"Remaining channels: {EEG.ch_names}")

filtered_eeg = signalfloweeg.preprocessing.filtering.bandpass_filter(EEG, 2, 50)
filtered_eeg = signalfloweeg.preprocessing.filtering.notch_filter(filtered_eeg, 60)
filtered_eeg = signalfloweeg.preprocessing.resampling.resample(filtered_eeg, 500)
epochs = signalfloweeg.preprocessing.epoching.epoch_time(filtered_eeg, 5)

# # Access the segmented data
# data = epochs.get_data()
# print(data.shape)  # Shape of the segmented data

# # Plot the epochs
# epochs.plot(block=True)


Reading /home/cbl/Desktop/Test_Data/0012_rest.fdt
Remaining channels: ['E3', 'E4', 'E5', 'E6', 'E7', 'E10', 'E11', 'E12', 'E13', 'E16', 'E18', 'E19', 'E20', 'E23', 'E24', 'E28', 'E29', 'E106', 'E111', 'E112', 'E117', 'E118', 'E124']
Reading 0 ... 307139  =      0.000 ...   307.139 secs...


/home/cbl/mne-python/1.7.0_0/lib/python3.12/site-packages/pymatreader/utils.py:238: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 1651 samples (1.651 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 58 - 62 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 58.00
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 57.75 Hz)
- Upper passband edg

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Not setting metadata
61 matching events found
No baseline correction applied
0 projection items activated


In [55]:
signalfloweeg.viz.heatmap.heatmap_power(epochs)

AttributeError: module 'signalfloweeg.viz.heatmap' has no attribute 'heatmap_power'